In [1]:
from paddleocr import PaddleOCR
import pandas as pd
import numpy as np
import unicodedata
import json
import cv2
import sys
import re
import os
from IPython.display import display, HTML

# Opcional: más columnas visibles en la salida
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 120)


print("Librerias Cargadas")

Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


Librerias Cargadas


In [2]:
#ocr = PaddleOCR(
#    use_doc_orientation_classify=False, 
 #   use_doc_unwarping=False, 
 #   use_textline_orientation=False) # text detection + text recognition

# text image preprocessing + text detection + textline orientation classification + text recognition
ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False, lang="es")

# ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False) # text detection + textline orientation classification + text recognition
# ocr = PaddleOCR(
#     text_detection_model_name="PP-OCRv5_mobile_det",
#     text_recognition_model_name="PP-OCRv5_mobile_rec",
#     use_doc_orientation_classify=False,
#     use_doc_unwarping=False,
#     use_textline_orientation=False) # Switch to PP-OCRv5_mobile models

/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


In [3]:
image_dir = "./imagenes"  # ruta a tu carpeta con imágenes
image_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# Filtramos solo imágenes
image_files = sorted(
    [os.path.join(image_dir, f) 
     for f in os.listdir(image_dir) 
     if os.path.splitext(f)[1].lower() in image_extensions],
    key=lambda x: int(os.path.splitext(os.path.basename(x.split("_")[1]))[0])
)

print(str(len(image_files))+" Imagenes Cargadas")

428 Imagenes Cargadas


In [4]:
def ocr_to_multidimensional_sections(result, line_gap=6.5, cortes=None):
    texts = result["rec_texts"]
    boxes = result["rec_boxes"]

    # Extraer coordenadas y asociar texto
    data = []
    for text, box in zip(texts, boxes):
        x_min, y_min = box[0], box[1]  # esquina superior izq
        text = clean_text_regex(text)
        data.append({"text": text, "x": x_min, "y": y_min})

    # Ordenar primero por Y (vertical) y luego por X (horizontal)
    data = sorted(data, key=lambda d: (d["y"], d["x"]))

    # Agrupamos por líneas (Y)
    ordered_lines = []
    current_line = []
    last_y = None

    for item in data:
        if last_y is None or abs(item["y"] - last_y) <= line_gap:
            current_line.append(item)
        else:
            ordered_lines.append(current_line)
            current_line = [item]
        last_y = item["y"]

    if current_line:
        ordered_lines.append(current_line)

    # Si no se pasan cortes, devolvemos normal
    if not cortes:
        max_len = max(len(line) for line in ordered_lines)
        ordered_filled = [
            [i["text"] for i in line] + [""] * (max_len - len(line))
            for line in ordered_lines
        ]
        return pd.DataFrame(ordered_filled)

    # --- Usar cortes en X para dividir en secciones ---
    cortes = sorted(cortes)  # [178, 286, 373] por ejemplo
    n_sections = len(cortes) + 1

    all_rows = []
    for line in ordered_lines:
        row = [""] * n_sections
        for item in line:
            x = item["x"]
            text = item["text"]

            # Buscar en qué sección cae
            section_idx = 0
            for c in cortes:
                if x > c:
                    section_idx += 1
                else:
                    break
            row[section_idx] += (" " + text if row[section_idx] else text)

        all_rows.append(row)

    df = pd.DataFrame(all_rows)
    return df

In [5]:
# Compilamos el patrón una sola vez (mejor rendimiento si llamas mucho a la función)
def _build_remove_regex(removes):
    """
    Genera un patrón que:
      - Ignora acentos (la función principal normaliza a ASCII).
      - Acepta separadores entre palabras (espacio, guion, underscore).
      - Permite variaciones de puntuación al final (... !!! ???).
      - Quita como token completo (no dentro de otras palabras).
      - Acepta pluralización simple: s | es (opcional).
    """
    parts = []
    for raw in removes:
        if not raw:
            continue
        # normalizar a ASCII (coincidir sin acentos)
        base = unicodedata.normalize("NFKD", raw).encode("ascii", "ignore").decode("ascii").lower().strip()
        if not base:
            continue

        # escapar y permitir separadores entre palabras (si hay espacios)
        # "guia rapida" -> "guia[\\s_\\-]+rapida"
        token = re.escape(base).replace(r"\ ", r"[\s_\-]+")

        # permitir puntos/símbolos finales repetidos
        # e.g., "continua..." -> "continua[\.\!\?\u2026]*"
        token = fr"{token}(?:[\-\.\!\?\u2026]+)?"

        # plural simple español (s|es) opcional cuando termina en letra
        # (solo añadimos si la última es alfanumérica)
        if re.search(r"[a-z0-9]$", base):
            token = fr"{token}(?:es|s)?"

        # asegurar “borde” de palabra a la española tras normalizar (ASCII):
        # que no esté pegado a letras/dígitos por los lados
        token = fr"(?<![A-Za-z0-9])(?:{token})(?![A-Za-z0-9])"
        parts.append(token)

    if not parts:
        # algo que nunca coincida
        return re.compile(r"(?!x)x", flags=re.IGNORECASE)

    big = "|".join(parts)
    return re.compile(big, flags=re.IGNORECASE)

# lista por defecto (puedes ampliarla)
_DEFAULT_REMOVES = [
    "continua...", "continua", "ejemplo", "borrar", "unidades", "nuevas" ,"suv",
    "linea","nueva","Usadas","Actualizacion", "UnidadesNuevas", "Precios", "de:"
    # agrega aquí más frases/palabras
]
_REMOVE_RE = _build_remove_regex(_DEFAULT_REMOVES)

def clean_text_regex(text, extra_removes=None):
    """
    Limpia 'text' eliminando tokens molestos con regex robusto.
    - extra_removes: lista adicional de términos/frases a quitar.
    """
    if not isinstance(text, str):
        return text

    # normaliza y elimina diacríticos (á->a, ñ->n)
    s = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("ascii")

    # quita comillas curvas/rectas y underscores sueltos repetidos
    s = re.sub(r"[\'\u2019_]+", "", s)

    # compila patrón combinado si hay extras
    if extra_removes:
        combined = list(_DEFAULT_REMOVES) + list(extra_removes)
        remove_re = _build_remove_regex(combined)
    else:
        remove_re = _REMOVE_RE

    # elimina tokens objetivo
    s = remove_re.sub(" ", s)

    # colapsa espacios y limpia
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [6]:
limite_indice = 9

In [7]:
def crear_imagenes_con_lineas(promedios, limite):
    output_dir = "./imagenes_con_lineas"
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in image_files[limite_indice:limite]:
        img = cv2.imread(img_path)
        if img is None:
            print(f" No se pudo leer: {img_path}")
            continue
    
        # Obtener dimensiones
        height, width, _ = img.shape
    
        # Dibujar cada línea
        for x in promedios:
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.line(img, (int(x), 0), (int(x), height), color, thickness)
    
        # Guardar nueva imagen
        output_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, img)


In [8]:
all_texts = {}
promedios = [100,800,970]

resp = input("¿Deseas continuar? (s/n): ").strip().lower()

if resp in ("s", "si", "y", "yes"):
    print("Usuario dijo SÍ")

    # Segunda pregunta
    raw = input("¿Cuántas páginas deseas procesar? (0 = todas): ").strip()
    try:
        num = int(raw)
    except ValueError:
        print("Número inválido:", raw)
        sys.exit(1)

    if num < 0:
        print("El número no puede ser negativo.")
        sys.exit(1)

    # si marca 0, toma todas las páginas
    if num == 0:
        num = len(image_files)
        print(f"Procesando todas las {num} páginas detectadas...")

    # limitar por si num > len(image_files)
    num = min(num, len(image_files))

    crear_imagenes_con_lineas(promedios, num)

    # recorrer solo las páginas solicitadas
    for img_path in image_files[limite_indice:num]:
        print(f"Procesando: {img_path}")
        res = ocr.predict(img_path)
        df_to_add = ocr_to_multidimensional_sections(
            res[0], line_gap=6.5, cortes=promedios
        )
        all_texts[os.path.basename(img_path)] = df_to_add

else:
    print("Usuario dijo NO")

¿Deseas continuar? (s/n):  s


Usuario dijo SÍ


¿Cuántas páginas deseas procesar? (0 = todas):  0


Procesando todas las 428 páginas detectadas...
Procesando: ./imagenes/img_010.jpg
Procesando: ./imagenes/img_011.jpg
Procesando: ./imagenes/img_012.jpg
Procesando: ./imagenes/img_013.jpg
Procesando: ./imagenes/img_014.jpg
Procesando: ./imagenes/img_015.jpg
Procesando: ./imagenes/img_016.jpg
Procesando: ./imagenes/img_017.jpg
Procesando: ./imagenes/img_018.jpg
Procesando: ./imagenes/img_019.jpg
Procesando: ./imagenes/img_020.jpg
Procesando: ./imagenes/img_021.jpg
Procesando: ./imagenes/img_022.jpg
Procesando: ./imagenes/img_023.jpg
Procesando: ./imagenes/img_024.jpg
Procesando: ./imagenes/img_025.jpg
Procesando: ./imagenes/img_026.jpg
Procesando: ./imagenes/img_027.jpg
Procesando: ./imagenes/img_028.jpg
Procesando: ./imagenes/img_029.jpg
Procesando: ./imagenes/img_030.jpg
Procesando: ./imagenes/img_031.jpg
Procesando: ./imagenes/img_032.jpg
Procesando: ./imagenes/img_033.jpg
Procesando: ./imagenes/img_034.jpg
Procesando: ./imagenes/img_035.jpg
Procesando: ./imagenes/img_036.jpg
Procesan

# Guardar todo en un solo Excel

In [9]:
def normalize(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s2 = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s2 = s2.lower().strip()
    s2 = re.sub(r"[^\w\s-]+", " ", s2)      # elimina signos
    s2 = re.sub(r"[_\s\-]+", " ", s2).strip()
    return s2

In [10]:
#cargar marcas
marcas_path = "marcas.csv"  # ajusta ruta si es necesario
marcas_df = pd.read_csv(marcas_path)

if "marca" not in marcas_df.columns:
    raise ValueError("El CSV de marcas debe tener una columna llamada 'marca'.")

# mapa normalizado -> original (para conservar capitalización)
marca_norm_to_original = {}
for m in marcas_df["marca"].dropna().astype(str):
    key = normalize(m)
    if key:
        marca_norm_to_original[key] = m.strip()

marca_norm_set = set(marca_norm_to_original.keys())

In [11]:
excel_path = "ocr_resultado_completo.xlsx"

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(all_texts.values(), ignore_index=True)

# Eliminar filas donde TODAS las columnas estén vacías o con espacios
final_df = final_df.replace(r'^\s*$', np.nan, regex=True)  # convierte espacios en NaN
final_df = final_df.dropna(how='all')  # elimina filas donde todo es NaN

final_df.to_excel("ocr_origen.xlsx", index=False, header=False)


In [12]:
def separar_anio_y_resto(texto):
    if not isinstance(texto, str):
        return np.nan, texto, False
    partes = texto.strip().split(maxsplit=1)
    if len(partes) == 0:
        return np.nan, texto, False
    primer = partes[0]
    resto = partes[1] if len(partes) > 1 else ""
    if primer.isdigit() and 1900 <= int(primer) <= 2099:
        return primer, resto , True
    return np.nan, texto, False


In [13]:
def show_df(df, title=None, cols=None, n=50):
    """Muestra un título y un snapshot del DataFrame (o columnas seleccionadas)."""
    if title:
        display(HTML(f"<h3 style='margin:8px 0'>{title}</h3>"))
    if cols is None:
        display(df.head(n))
    else:
        # Muestra solo columnas existentes (evita KeyError si alguna falta)
        cols = [c for c in cols if c in df.columns]
        display(df[cols].head(n))

# ------------------------------------------------------------------
# 2) Copiamos y mostramos
# ------------------------------------------------------------------
df = final_df.copy()

# ------------------------------------------------------------------
# 3) Detectar año y resto desde la columna 1 (como en tu flujo)
# ------------------------------------------------------------------
anio_resto = df[1].apply(lambda x: pd.Series(separar_anio_y_resto(x)))

if "version" not in df.columns:
    df["version"] = pd.Series(pd.NA, index=df.index)

df["version"] = df["version"].ffill()

df["año"]   = anio_resto[0]
df["texto"] = anio_resto[1]

# ------------------------------------------------------------------
# 4) Propagar el año hacia abajo (ffill)
# ------------------------------------------------------------------
df["año"] = df["año"].ffill()

# ------------------------------------------------------------------
# 5) Detectar modelo y versión (tu lógica base)
#    - texto corto (<=3 tokens) -> modelo
#    - texto largo -> versión del modelo actual
# ------------------------------------------------------------------
modelos = []
versiones = []
modelo_actual = None

for texto in df["texto"]:
    if isinstance(texto, str) and texto.strip() != "":
        if len(texto.split()) <= 3:  # criterio: texto corto → modelo
            modelo_actual = texto.strip()
            modelos.append(modelo_actual)
            versiones.append(np.nan)
        else:
            modelos.append(modelo_actual)
            versiones.append(texto.strip())
    else:
        modelos.append(modelo_actual)
        versiones.append(np.nan)

df["modelo"]  = modelos
df["version"] = versiones

# ------------------------------------------------------------------
# 6) Agregar columna 'marca' (después de año) y 'id' (después de version)
#    Aquí las creamos vacías; puedes llenarlas luego.
# ------------------------------------------------------------------
df = df.copy()  # tu DataFrame de trabajo que ya tiene la columna "texto"
df["texto_norm"] = df["texto"].apply(normalize)

# ¿la fila es exactamente una marca?
df["marca_key"] = df["texto_norm"].where(df["texto_norm"].isin(marca_norm_set))

# asigna el nombre original de la marca donde corresponde
df["marca_detectada"] = df["marca_key"].map(marca_norm_to_original)

# propaga la marca hacia abajo: TODAS las filas tendrán marca (desde la primera detección)
df["marca"] = df["marca_detectada"].ffill()

df["id"]    = np.nan
#show_df(df, "5) Agregadas columnas 'marca' e 'id'", cols=["año", "marca", "modelo", "version", "id"])


# ------------------------------------------------------------------
# 7) Reordenar columnas: año, marca, modelo, version, id, (resto)
# ------------------------------------------------------------------
columnas_ordenadas = (
    ["año", "marca", "modelo", "version", "id"]
    + [c for c in df.columns if c not in [0, 1, "texto", "año", "marca", "modelo", "version", "id"]]
)
excel_df = df[columnas_ordenadas]

#display(excel_df)

# ------------------------------------------------------------------
# 8) Guardar a Excel (opcional)
# ------------------------------------------------------------------
excel_path = "salida_modelos_versiones.xlsx"  # ajusta ruta si lo deseas
excel_df.to_excel(excel_path, index=False, header=False)